** T4에서 실행 안됨. L4이상부터 가능
# FlashAttention
- 연산 속도를 빠르게 할 수 있는 방법
- 기존 self attention 연산을 할 때 데이터를 메모리에서 빠르게 읽고 쓰는 데 병목현상이 있음

## FlashAttention-2
- FlashAttentio에서의 forward 및 backward 알고리즘을 수정해 연산속도를 개선한 알고리즘
- FlashAttention 대비 2배 빠른 속도

In [ ]:
!pip install flash-attn --no-build-isolation
!pip install accelerate
!pip install transformers==4.39.3

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# phi-2 with flash attention2
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM
)

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
    trust_remote_code=True
)

In [ ]:
prompt = """ def factorial(n):
Calcuate the factorial of a number n
"""

input_ids = tokenizer(
    prompt,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id
]

In [ ]:
outputs = model.generate(
    **input_ids, # dictionary 형태 파싱
    max_new_tokens=200,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
response = outputs[0][input_ids['input_ids'].shape[-1]:]

print(tokenizer.decode(response, skip_special_tokens=True))